In [1]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [2]:
#from ktrain import text
import pandas as pd
import json

with open('Musical_Instruments_5.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

df = pd.DataFrame(data)

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [4]:
df["target"]= df["overall"].astype('category').cat.codes

In [5]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,target
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014",4
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013",4
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013",4
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",4
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014",4


In [6]:
df["target"].value_counts()/len(df["target"])*100

4    67.615242
3    20.309911
2     7.523633
1     2.436410
0     2.114804
Name: target, dtype: float64

In [7]:
df["target"].value_counts()/len(df["overall"])*100

4    67.615242
3    20.309911
2     7.523633
1     2.436410
0     2.114804
Name: target, dtype: float64

In [8]:
X=list(df['reviewText'])
y=list(df['target'])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
y_test

In [10]:
!pip install transformers
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

     |████████████████████████████████| 2.3MB 8.0MB/s 
     |████████████████████████████████| 3.3MB 38.6MB/s 
     |████████████████████████████████| 901kB 47.5MB/s 


In [11]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [14]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [16]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_39', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [19]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=2, batch_size=16, validation_data=test_dataset.shuffle(1000).batch(16))


Epoch 1/2
513/513 [==============================] - 493s 962ms/step - loss: 0.8136 - accuracy: 0.6932 - val_loss: 0.7440 - val_accuracy: 0.6863
Epoch 2/2
513/513 [==============================] - 502s 978ms/step - loss: 0.6355 - accuracy: 0.7443 - val_loss: 0.7937 - val_accuracy: 0.6902


In [36]:
import tensorflow as tf
tf.argmax(model.predict(test_dataset).logits,axis= 1).numpy()

array([4, 4, 4, ..., 4, 4, 4])

In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score

cm=confusion_matrix(y_test,output)
cm


ValueError: ignored

In [40]:
accuracy_score(y_test,output)

1.0

In [20]:
save_directory = "/content" # change this to your preferred location

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/vocab.txt',
 '/content/added_tokens.json',
 '/content/tokenizer.json')

In [ ]:
logits = outputs.logits
